In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import torch as torch
import nltk
from datasets import load_dataset
import torch 

torch.cuda.device_count()

0

In [5]:
dataset = load_dataset('csv', data_files = 'spotify_millsongdata.csv')

In [6]:
from transformers import GPT2Tokenizer, GPT2Model
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from transformers import GPT2TokenizerFast


model = GPT2Model.from_pretrained('gpt2-medium')
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

In [7]:
encoded_input = tokenizer("melodie gen abba", return_tensors="pt")
encoded_input


{'input_ids': tensor([[17694,   375,   494,  2429,   450,  7012]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [48]:
MAX_LENGTH = 256
BATCH_SIZE = 6
tokenized_data = tokenizer(
    dataset['train']['text'],
    truncation = True,
    max_length = MAX_LENGTH,
    return_overflowing_tokens = True,
    return_length = True
)
tokenized_data = {"input_ids": tokenized_data["input_ids"]}
tokenized_data

KeyboardInterrupt: 

In [49]:
def preprocess(example):
    try:
        outputs = tokenizer(
        example["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        return_overflowing_tokens=True,
        return_length=True,
        padding="max_length",  # Aceasta va utiliza pad_token implicit
        return_tensors="pt"
    )

        
        input_batch = outputs["input_ids"].tolist()

        if len(input_batch) < BATCH_SIZE:
            input_batch += [input_batch[0]] * (BATCH_SIZE - len(input_batch))
    except Exception as e:
        print(f"An error occurred: {e}")
        input_batch = []

    return {"input_ids": input_batch}


tokenized_dataset = dataset.map(preprocess, remove_columns = dataset['train'].column_names)


Map:   0%|          | 0/57650 [00:00<?, ? examples/s]

In [51]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 57650
    })
})

In [52]:
def filter_none(example):
    if len(example['input_ids']) >= 1:
        return example
final_data = tokenized_dataset.filter(filter_none)


Filter:   0%|          | 0/57650 [00:00<?, ? examples/s]

In [56]:
tokenizer.pad_token = tokenizer.eos_token
data_col = DataCollatorForLanguageModeling(tokenizer, mlm = False, return_tensors = "tf") # mlm = masked language modeling

tf_train_dataset = final_data["train"].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'labels'],
    shuffle = True,
    batch_size = 1,
    collate_fn = data_col
)

In [59]:
import tensorflow as tf

def swap_position(example):
    return {
        'input_ids': example['input_ids'],
        'attention_mask': tf.ones([1, BATCH_SIZE, MAX_LENGTH]),
        'labels': example['labels']
    }
train_data = tf_train_dataset.map(swap_position)
unbatched_train_data = train_data.unbatch()


In [60]:
from transformers import TFGPT2LMHeadModel
model = TFGPT2LMHeadModel.from_pretrained('gpt2-medium')
model.summary()

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  354823168 
 er)                                                             
                                                                 
Total params: 354823168 (1.32 GB)
Trainable params: 354823168 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
from transformers import create_optimizer
num_train_steps = len(unbatched_train_data)
optimizer, schedule = create_optimizer(
    init_lr = 5e-5,
    num_warmup_steps = 1_000,
    num_train_steps = num_train_steps
)

model.compile(
    optimizer = optimizer
)


In [ ]:
train_history = model.fit(unbatched_train_data, epochs = 5)


Epoch 1/5

